In [49]:

import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, optimizers, metrics
from tensorflow.keras.layers import Input, Conv1D, GlobalAveragePooling1D, Dense, Dropout, BatchNormalization, Concatenate, Lambda
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import sklearn.metrics


In [50]:

data = pd.read_csv('../Data/freesolv.csv')
data_clean = data.dropna(subset=['smiles'])
smiles = data_clean['smiles']
labels = data_clean['freesolv'].astype(float).values


In [51]:

smiles_dict = {
    "#": 29, "%": 30, ")": 31, "(": 1, "+": 32, "-": 33, "/": 34, ".": 2,
    "1": 35, "0": 3, "3": 36, "2": 4, "5": 37, "4": 5, "7": 38, "6": 6,
    "9": 39, "8": 7, "=": 40, "A": 41, "@": 8, "C": 42, "B": 9, "E": 43,
    "D": 10, "G": 44, "F": 11, "I": 45, "H": 12, "K": 46, "M": 47, "L": 13,
    "O": 48, "N": 14, "P": 15, "S": 49, "R": 16, "U": 50,
}
MAX_SMI_LEN = 100
NUM_CHARS = max(smiles_dict.values()) + 1
XD = np.zeros((len(smiles), MAX_SMI_LEN, NUM_CHARS))
for i, smi in enumerate(smiles):
    for j, ch in enumerate(smi[:MAX_SMI_LEN]):
        XD[i, j, smiles_dict.get(ch, 0)] = 1


In [52]:

XDinput = Input(shape=(MAX_SMI_LEN, NUM_CHARS), name='XDinput')
encode_smiles = Conv1D(filters=64, kernel_size=2, activation='relu')(XDinput)
encode_smiles = Conv1D(filters=64, kernel_size=4, activation='relu')(encode_smiles)
encode_smiles = Conv1D(filters=128, kernel_size=4, activation='relu')(encode_smiles)
model_feature = keras.Model(inputs=XDinput, outputs=encode_smiles, name='model_feature')


In [53]:

class TopkRouting(layers.Layer):
    def __init__(self, qk_dim, topk=16, qk_scale=None):
        super().__init__()
        self.topk = topk
        self.qk_dim = qk_dim
        self.scale = qk_scale if qk_scale is not None else qk_dim**-0.5
        self.to_q = layers.Dense(qk_dim)
        self.to_k = layers.Dense(qk_dim)
        self.to_v = layers.Dense(qk_dim)
    def call(self, x):
        q = self.to_q(x)
        k = self.to_k(x)
        v = self.to_v(x)
        attn = tf.matmul(q, k, transpose_b=True) * self.scale
        topk = tf.nn.top_k(attn, k=self.topk).indices
        gather = tf.gather(v, topk, batch_dims=1)
        return tf.reduce_mean(gather, axis=2)

class Block(layers.Layer):
    def __init__(self, dim, num_heads=8, topk=16, mlp_ratio=3):
        super().__init__()
        self.attn = TopkRouting(dim, topk)
        self.norm1 = layers.LayerNormalization()
        self.mlp = keras.Sequential([
            layers.Dense(dim*mlp_ratio, activation='relu'),
            layers.Dense(dim)
        ])
        self.norm2 = layers.LayerNormalization()
    def call(self, x):
        h = self.attn(x)
        x = self.norm1(x + h)
        h2 = self.mlp(x)
        return self.norm2(x + h2)


In [54]:

dim = 128
transformer_block1 = Block(dim)
transformer_block2 = Block(dim)

input_inter = Input(shape=(MAX_SMI_LEN, NUM_CHARS))
features = model_feature(input_inter)
trans1 = transformer_block1(features)
trans1 = Lambda(lambda x: tf.norm(x, axis=-1, keepdims=True))(trans1)
trans2 = transformer_block2(features)
trans2 = Lambda(lambda x: tf.norm(x, axis=-1, keepdims=True))(trans2)
combined = Concatenate(axis=-1)([trans1, trans2])
pooled = GlobalAveragePooling1D()(combined)
difference = Lambda(lambda x: tf.expand_dims(x[:,0]-x[:,1], axis=-1))(pooled)
interactionModel = keras.Model(inputs=input_inter, outputs=difference, name='interactionModel')
interactionModel.summary()


Model: "interactionModel"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_15      │ (None, 100, 51)   │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ model_feature       │ (None, 93, 128)   │     55,936 │ input_layer_15[0… │
│ (Functional)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_24 (Block)    │ (None, 93, 128)   │    148,864 │ model_feature[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_25 (Block)    │ (None, 93, 128)   │    148,864 │ model_feature[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lambda_14 (Lambda)  │ (None, 93, 1)     │          0 │ block_24[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lambda_15 (Lambda)  │ (None, 93, 1)     │          0 │ block_25[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_5       │ (None, 93, 2)     │          0 │ lambda_14[0][0],  │
│ (Concatenate)       │                   │            │ lambda_15[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_average_poo… │ (None, 2)         │          0 │ concatenate_5[0]… │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lambda_16 (Lambda)  │ (None, 1)         │          0 │ global_average_p… │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 353,664 (1.35 MB)

 Trainable params: 353,664 (1.35 MB)

 Non-trainable params: 0 (0.00 B)

In [55]:

XD_np = np.array(XD)
labels_np = labels
index = np.where(~np.isnan(labels_np))
XD_np = XD_np[index]
labels_np = labels_np[index]
X_train, X_test, y_train, y_test = train_test_split(XD_np, labels_np, test_size=0.2, random_state=9)
scaler_y = StandardScaler()
y_train = scaler_y.fit_transform(y_train.reshape(-1,1))
y_test = scaler_y.transform(y_test.reshape(-1,1))


In [56]:

METRICS_REGRESSION = [metrics.MeanAbsoluteError(name='mae'), metrics.MeanSquaredError(name='mse')]
class R2Callback(keras.callbacks.Callback):
    def __init__(self, data):
        super().__init__()
        self.x, self.y = data
    def on_epoch_end(self, epoch, logs=None):
        y_pred = self.model.predict(self.x, verbose=0)
        r2 = sklearn.metrics.r2_score(self.y, y_pred)
        logs = logs or {}
        logs['r2_score'] = r2
        print(f" - r2_score: {r2:.4f}")

opt = optimizers.Adam(learning_rate=1e-4)
interactionModel.compile(optimizer=opt, loss='mse', metrics=METRICS_REGRESSION)
cb = [keras.callbacks.EarlyStopping(monitor='loss', patience=30, restore_best_weights=True), R2Callback((X_train, y_train))]
interactionModel.fit(X_train, y_train, batch_size=256, epochs=100
                     , callbacks=cb, verbose=1)


Epoch 1/100


c:\Users\Scott.Coffin\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\optimizers\base_optimizer.py:855: UserWarning: Gradients do not exist for variables ['block_24/topk_routing_24/dense_124/kernel', 'block_24/topk_routing_24/dense_124/bias', 'block_24/topk_routing_24/dense_125/kernel', 'block_24/topk_routing_24/dense_125/bias', 'block_25/topk_routing_25/dense_129/kernel', 'block_25/topk_routing_25/dense_129/bias', 'block_25/topk_routing_25/dense_130/kernel', 'block_25/topk_routing_25/dense_130/bias'] when minimizing the loss. If using `model.compile()`, did you forget to provide a `loss` argument?
  warnings.warn(


2/3 ━━━━━━━━━━━━━━━━━━━━ 0s 500ms/step - loss: 14.3019 - mae: 2.2966 - mse: 14.3019 - r2_score: -12.5623
3/3 ━━━━━━━━━━━━━━━━━━━━ 6s 781ms/step - loss: 20.8999 - mae: 3.0548 - mse: 20.8999 - r2_score: -12.5623
Epoch 2/100
2/3 ━━━━━━━━━━━━━━━━━━━━ 0s 443ms/step - loss: 12.1768 - mae: 3.0011 - mse: 12.1768 - r2_score: -1.7647
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 481ms/step - loss: 11.2369 - mae: 2.8595 - mse: 11.2369 - r2_score: -1.7647
Epoch 3/100
2/3 ━━━━━━━━━━━━━━━━━━━━ 0s 442ms/step - loss: 2.5651 - mae: 1.3041 - mse: 2.5651 - r2_score: -0.5247
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 487ms/step - loss: 2.5102 - mae: 1.2888 - mse: 2.5102 - r2_score: -0.5247
Epoch 4/100
2/3 ━━━━━━━━━━━━━━━━━━━━ 0s 548ms/step - loss: 1.4932 - mae: 0.9970 - mse: 1.4932 - r2_score: -0.2258
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 615ms/step - loss: 1.4743 - mae: 0.9797 - mse: 1.4743 - r2_score: -0.2258
Epoch 5/100
2/3 ━━━━━━━━━━━━━━━━━━━━ 0s 504ms/step - loss: 1.2800 - mae: 0.8811 - mse: 1.2800 - r2_score: -0.0969
3/3 ━━━━━━━━━━━━━━━━━━━━

In [57]:

new_transformer_block = Block(dim)
new_transformer_block2 = Block(dim)
dummy = tf.zeros((1,) + model_feature.output_shape[1:])
new_transformer_block(dummy)
new_transformer_block2(dummy)
new_transformer_block.set_weights(transformer_block1.get_weights())
new_transformer_block2.set_weights(transformer_block2.get_weights())
model_feature.trainable = False
new_transformer_block.trainable = False
new_transformer_block2.trainable = False
input_final = Input(shape=(MAX_SMI_LEN, NUM_CHARS))
feat_out = model_feature(input_final)
tr1 = new_transformer_block(feat_out)
tr1 = Lambda(lambda x: tf.norm(x, axis=-1, keepdims=True))(tr1)
tr2 = new_transformer_block2(feat_out)
tr2 = Lambda(lambda x: tf.norm(x, axis=-1, keepdims=True))(tr2)
concat = Concatenate(axis=-1)([tr1, tr2])
pooled_f = GlobalAveragePooling1D()(concat)
fc1 = Dense(512, activation='relu')(pooled_f)
fc1 = BatchNormalization()(fc1)
fc1 = Dropout(0.1)(fc1)
fc2 = Dense(256, activation='relu')(fc1)
fc2 = BatchNormalization()(fc2)
fc2 = Dropout(0.1)(fc2)
fc3 = Dense(64, activation='relu')(fc2)
output = Dense(1)(fc3)
model_final = keras.Model(inputs=input_final, outputs=output, name='regression_model')
model_final.summary()


Model: "regression_model"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_20      │ (None, 100, 51)   │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ model_feature       │ (None, 93, 128)   │     55,936 │ input_layer_20[0… │
│ (Functional)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_26 (Block)    │ (None, 93, 128)   │    148,864 │ model_feature[1]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_27 (Block)    │ (None, 93, 128)   │    148,864 │ model_feature[1]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lambda_17 (Lambda)  │ (None, 93, 1)     │          0 │ block_26[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lambda_18 (Lambda)  │ (None, 93, 1)     │          0 │ block_27[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_6       │ (None, 93, 2)     │          0 │ lambda_17[0][0],  │
│ (Concatenate)       │                   │            │ lambda_18[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_average_poo… │ (None, 2)         │          0 │ concatenate_6[0]… │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_144 (Dense)   │ (None, 512)       │      1,536 │ global_average_p… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 512)       │      2,048 │ dense_144[0][0]   │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_2 (Dropout) │ (None, 512)       │          0 │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_145 (Dense)   │ (None, 256)       │    131,328 │ dropout_2[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 256)       │      1,024 │ dense_145[0][0]   │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_3 (Dropout) │ (None, 256)       │          0 │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_146 (Dense)   │ (None, 64)        │     16,448 │ dropout_3[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_147 (Dense)   │ (None, 1)         │         65 │ dense_146[0][0]   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 506,113 (1.93 MB)

 Trainable params: 150,913 (589.50 KB)

 Non-trainable params: 355,200 (1.35 MB)

In [58]:

opt_final = optimizers.Adam(learning_rate=1e-4)
model_final.compile(optimizer=opt_final, loss='mse', metrics=METRICS_REGRESSION)
cb_final = [keras.callbacks.EarlyStopping(monitor='loss', patience=30, restore_best_weights=True), R2Callback((X_train, y_train))]
model_final.fit(X_train, y_train, batch_size=256, epochs=100, callbacks=cb_final, verbose=1)


Epoch 1/100
2/3 ━━━━━━━━━━━━━━━━━━━━ 0s 185ms/step - loss: 1.5178 - mae: 0.7197 - mse: 0.9932 - r2_score: 0.0167
3/3 ━━━━━━━━━━━━━━━━━━━━ 5s 946ms/step - loss: 1.5062 - mae: 0.7361 - mse: 1.0258 - r2_score: 0.0167
Epoch 2/100
2/3 ━━━━━━━━━━━━━━━━━━━━ 0s 223ms/step - loss: 0.9845 - mae: 0.7753 - mse: 0.9845 - r2_score: 0.0162
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 457ms/step - loss: 1.0576 - mae: 0.7965 - mse: 1.0576 - r2_score: 0.0162
Epoch 3/100
2/3 ━━━━━━━━━━━━━━━━━━━━ 0s 195ms/step - loss: 1.0104 - mae: 0.7657 - mse: 1.0104 - r2_score: 0.0107
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 394ms/step - loss: 1.0090 - mae: 0.7674 - mse: 1.0090 - r2_score: 0.0107
Epoch 4/100
2/3 ━━━━━━━━━━━━━━━━━━━━ 0s 205ms/step - loss: 1.1375 - mae: 0.7910 - mse: 1.1375 - r2_score: 0.0099
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 386ms/step - loss: 1.1186 - mae: 0.7822 - mse: 1.1186 - r2_score: 0.0099
Epoch 5/100
2/3 ━━━━━━━━━━━━━━━━━━━━ 0s 197ms/step - loss: 0.9497 - mae: 0.7361 - mse: 0.9497 - r2_score: 0.0129
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 389

In [61]:

def evaluate(model, X, y_scaled, y_orig):
    y_pred_scaled = model.predict(X)
    y_pred = scaler_y.inverse_transform(y_pred_scaled)
    mae = sklearn.metrics.mean_absolute_error(y_orig, y_pred)
    mse = sklearn.metrics.mean_squared_error(y_orig, y_pred)
    r2 = sklearn.metrics.r2_score(y_orig, y_pred)
    return mae, mse, r2

train_metrics = evaluate(model_final, X_train, y_train, scaler_y.inverse_transform(y_train))
test_metrics = evaluate(model_final, X_test, y_test, scaler_y.inverse_transform(y_test))
print("Train MAE, MSE, R2:", train_metrics)
print("Test MAE, MSE, R2:", test_metrics)


17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
Train MAE, MSE, R2: (2.152675839353491, 8.874838401350772, 0.3470302650197091)
Test MAE, MSE, R2: (2.5717974489788675, 14.842648498080848, 0.23182204666388984)
